In [1]:
import xml.etree.ElementTree as ET

def count_vertical_bars(xml_file, N):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Find all occurrences of <Hsp_midline>
    midlines = root.findall(".//Hsp_midline")
    
    # Check if N is within the bounds
    if N > len(midlines) or N < 1:
        raise ValueError(f"N={N} is out of bounds. There are only {len(midlines)} occurrences.")
    
    # Get the Nth occurrence (adjust for zero-based indexing)
    midline_text = midlines[N-1].text
    
    # Count the number of vertical bars '|'
    vertical_bar_count = midline_text.count('|')
    
    return vertical_bar_count

# Example usage:
# Assuming the XML file is named 'example.xml' and you want to find the number of vertical bars in the N <Hsp_midline> occurrence.
vertical_bar_count = count_vertical_bars('contigs.xml', 590)
print(f"Number of vertical bars in the N occurrence: {vertical_bar_count}")


Number of vertical bars in the N occurrence: 68


In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_xml_and_calculate(xml_file):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Initialize the dictionary to hold the results
    result_dict = {
        "id": [],
        "Hsp_align_len": [],
        "num_vertical_bars": [],
        "division_result": []
    }

    # Iterate over all occurrences of <Hsp>
    for idx, hsp_elem in enumerate(root.findall('.//Hsp')):
        hsp_align_len_elem = hsp_elem.find('Hsp_align-len')
        hsp_midline_elem = hsp_elem.find('Hsp_midline')
        
        if hsp_align_len_elem is not None and hsp_midline_elem is not None:
            hsp_align_len = int(hsp_align_len_elem.text)
            hsp_midline = hsp_midline_elem.text
            num_vertical_bars = hsp_midline.count('|')
            
            # Avoid division by zero
            if num_vertical_bars > 0:
                division_result = (num_vertical_bars / hsp_align_len) * 100
            else:
                division_result = None
            
            # Append to the result dictionary
            result_dict["id"].append(idx)
            result_dict["Hsp_align_len"].append(hsp_align_len)
            result_dict["num_vertical_bars"].append(num_vertical_bars)
            result_dict["division_result"].append(division_result)
        else:
            # Handle cases where elements are missing
            result_dict["id"].append(idx)
            result_dict["Hsp_align_len"].append(None if hsp_align_len_elem is None else int(hsp_align_len_elem.text))
            result_dict["num_vertical_bars"].append(0)
            result_dict["division_result"].append(None)

    # Convert the result dictionary to a DataFrame
    df = pd.DataFrame(result_dict)
    df.set_index("id", inplace=True)

    return df


In [3]:
df = parse_xml_and_calculate("example.xml")

In [4]:
#df = df.sort_values(by=['division_result','Hsp_align_len'], ascending = [False,False])


In [5]:
df

,Hsp_align_len,num_vertical_bars,division_result
id,,,
0,4524298,4524285,99.999713
1,5909,5884,99.576917
2,5909,5884,99.576917
3,5812,5795,99.707502
4,5812,5795,99.707502
...,...,...,...
815,28,28,100.000000
816,41,37,90.243902
817,28,28,100.000000


In [6]:
import xml.etree.ElementTree as ET
import pandas as pd

def extract_hsp_positions(xml_file):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Initialize a list to hold the results
    results = []
    
    # Iterate over all occurrences of <Hsp_num>
    for hsp_num_elem in root.findall('.//Hsp_num'):
        # Find the parent <Hsp> element
        hsp_elem = hsp_num_elem.getparent()
        
        # Extract the required elements
        hsp_query_from = hsp_elem.findtext('Hsp_query-from')
        hsp_query_to = hsp_elem.findtext('Hsp_query-to')
        hsp_hit_from = hsp_elem.findtext('Hsp_hit-from')
        hsp_hit_to = hsp_elem.findtext('Hsp_hit-to')
        
        # Append the extracted data to the results list
        results.append({
            "Hsp_num": int(hsp_num_elem.text),
            "Hsp_query_from": int(hsp_query_from) if hsp_query_from else None,
            "Hsp_query_to": int(hsp_query_to) if hsp_query_to else None,
            "Hsp_hit_from": int(hsp_hit_from) if hsp_hit_from else None,
            "Hsp_hit_to": int(hsp_hit_to) if hsp_hit_to else None,
        })
    
    # Convert the results list to a DataFrame
    df = pd.DataFrame(results)
    df.set_index("Hsp_num", inplace=True)

    return df


In [7]:
import xml.etree.ElementTree as ET
import pandas as pd

def extract_hsp_positions(xml_file):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Initialize a list to hold the results
    results = []
    
    # Iterate over all occurrences of <Hsp> elements
    for hsp_elem in root.findall('.//Hsp'):
        # Extract the <Hsp_num>
        hsp_num_elem = hsp_elem.find('Hsp_num')
        hsp_num = int(hsp_num_elem.text) if hsp_num_elem is not None else None
        
        # Extract the required elements
        hsp_query_from = hsp_elem.findtext('Hsp_query-from')
        hsp_query_to = hsp_elem.findtext('Hsp_query-to')
        hsp_hit_from = hsp_elem.findtext('Hsp_hit-from')
        hsp_hit_to = hsp_elem.findtext('Hsp_hit-to')
        
        # Append the extracted data to the results list
        results.append({
            "Hsp_num": hsp_num,
            "Hsp_query_from": int(hsp_query_from) if hsp_query_from else None,
            "Hsp_query_to": int(hsp_query_to) if hsp_query_to else None,
            "Hsp_hit_from": int(hsp_hit_from) if hsp_hit_from else None,
            "Hsp_hit_to": int(hsp_hit_to) if hsp_hit_to else None,
        })
    
    # Convert the results list to a DataFrame
    df = pd.DataFrame(results)
    df.set_index("Hsp_num", inplace=True)

    return df


In [8]:
extract_hsp_positions("example.xml")

,Hsp_query_from,Hsp_query_to,Hsp_hit_from,Hsp_hit_to
Hsp_num,,,,
1,367,4524660,1,4524289
2,3517912,3523818,2443528,2449436
3,2443901,2449808,3517540,3523447
4,3518010,3523818,725635,719824
5,720193,726003,3523447,3517638
...,...,...,...,...
816,895209,895236,62314,62287
817,62653,62692,894868,894829
818,2511952,2511979,1157981,1157954


In [9]:
extract_hsp_positions("contigs.xml")

,Hsp_query_from,Hsp_query_to,Hsp_hit_from,Hsp_hit_to
Hsp_num,,,,
1,94669,103533,151122,159977
2,86886,90408,140848,144384
3,108127,110989,164660,167516
4,83476,85402,138100,140032
5,63962,66386,116706,119132
...,...,...,...,...
9,131033,131937,201849,200945
1,93585,95181,4320,2724
1,18,132,115,1


In [15]:
import pandas as pd

def process_fasta(fasta_file):
    headers = []
    lengths = []
    start_positions = []
    end_positions = []

    total_length = 0
    start = 1

    with open(fasta_file, 'r') as file:
        sequence = ""
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if sequence:  # If there is a sequence, process it
                    length = len(sequence)
                    total_length += length
                    end = start + length - 1
                    start_positions.append(start)
                    end_positions.append(end)
                    start = end + 1
                    sequence = ""
                
                # Extract the header part between '>' and the first space
                header = line[1:].split()[0]
                headers.append(header)
            else:
                sequence += line

        # Process the last sequence
        if sequence:
            length = len(sequence)
            total_length += length
            end = start + length - 1
            start_positions.append(start)
            end_positions.append(end)

    # Fill the Length column with the total length of all sequences
    lengths = [total_length] * len(headers)

    # Create DataFrame
    df = pd.DataFrame({
        'Header': headers,
        'Total_Length': lengths,
        'Start': start_positions,
        'End': end_positions
    })

    return df


In [16]:
fasta_file = '10.fna'
df = process_fasta(fasta_file)
df

,Header,Total_Length,Start,End
0,NZ_JAEPPV010000008.1,3327926,1,118776
1,NZ_JAEPPV010000006.1,3327926,118777,335577
2,NZ_JAEPPV010000007.1,3327926,335578,460596
3,NZ_JAEPPV010000003.1,3327926,460597,921564
4,NZ_JAEPPV010000010.1,3327926,921565,1003997
5,NZ_JAEPPV010000004.1,3327926,1003998,1243580
6,NZ_JAEPPV010000009.1,3327926,1243581,1332050
7,NZ_JAEPPV010000001.1,3327926,1332051,2452878
8,NZ_JAEPPV010000002.1,3327926,2452879,3091883
9,NZ_JAEPPV010000005.1,3327926,3091884,3327926


In [2]:
def concatenate_fna_sequences(input_file):
    output_file = input_file.replace('.fna', '_concatenated.fna')
    
    with open(input_file, 'r') as infile:
        lines = infile.readlines()
    
    # Prepare to concatenate sequences
    concatenated_sequence = ""
    sequence_name = ""
    sequence = ""
    
    for line in lines:
        line = line.strip()
        if line.startswith('>'):
            # If a sequence name is found and we already have a sequence, save it
            if sequence_name:
                concatenated_sequence += sequence
            sequence_name = line[1:]  # Remove '>'
            sequence = ""
        else:
            sequence += line
    
    # Don't forget to add the last sequence
    if sequence_name:
        concatenated_sequence += sequence
    
    # Write the concatenated sequence to the output file
    with open(output_file, 'w') as outfile:
        outfile.write(f">{input_file.replace('.fna', '')}\n")
        outfile.write(concatenated_sequence + '\n')
    
    # Return and print the output filename
    print(f"Concatenated sequence written to {output_file}")
    return output_file

# Example usage:
# filename = concatenate_fna_sequences('your_file.fna')


In [4]:
filename = concatenate_fna_sequences('10.fna')

Concatenated sequence written to 10_concatenated.fna


In [5]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from ftplib import FTP
import gzip
import subprocess
import shutil

# Define the target download directory
download_directory = '/Users/niteshku/Notebooks/metagenomics/grant/'

# Sample DataFrame with FTP paths and genome_ids
data = {
    'ftp_paths': [
        'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/949/281/945/GCF_949281945.1_PT33-3/',
        'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/008/685/GCF_000008685.1_ASM868v1/'
    ],
    'genome_id': [
        '24.8',
        'GCF_000008685.1_ASM868v1'
    ]
}
df = pd.DataFrame(data)

def concatenate_fna_sequences(input_file):
    output_file = input_file.replace('.fna', '_concatenated.fna')
    
    with open(input_file, 'r') as infile:
        lines = infile.readlines()
    
    # Prepare to concatenate sequences
    concatenated_sequence = ""
    sequence_name = ""
    sequence = ""
    
    for line in lines:
        line = line.strip()
        if line.startswith('>'):
            # If a sequence name is found and we already have a sequence, save it
            if sequence_name:
                concatenated_sequence += sequence
            sequence_name = line[1:]  # Remove '>'
            sequence = ""
        else:
            sequence += line
    
    # Don't forget to add the last sequence
    if sequence_name:
        concatenated_sequence += sequence
    
    # Write the concatenated sequence to the output file
    with open(output_file, 'w') as outfile:
        outfile.write(f">{input_file.replace('.fna', '')}\n")
        outfile.write(concatenated_sequence + '\n')
    
    # Return and print the output filename
    print(f"Concatenated sequence written to {output_file}")
    return output_file

def download_and_process_files(ftp_url, genome_id, local_dir=download_directory):
    # Send a GET request to the FTP directory
    response = requests.get(ftp_url)
    
    # Parse the HTML response to extract links to files
    soup = BeautifulSoup(response.text, 'html.parser')
    files = [link.get('href') for link in soup.find_all('a')]

    # Find the file that ends with '_genomic.fna.gz' and does not contain 'cds' or 'rna'
    genomic_file = next((f for f in files if f.endswith('_genomic.fna.gz') and 'cds' not in f and 'rna' not in f), None)
    
    if genomic_file:
        # Ensure local directory exists
        os.makedirs(local_dir, exist_ok=True)
        local_gz_path = os.path.join(local_dir, genomic_file)
        local_fna_path = local_gz_path.replace('.gz', '')

        # Check if the file has already been downloaded
        if not os.path.exists(local_gz_path):
            file_url = os.path.join(ftp_url, genomic_file)

            # Download the file using requests
            with requests.get(file_url, stream=True) as r:
                with open(local_gz_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f'Downloaded: {genomic_file} to {local_gz_path}')
        else:
            print(f'File {genomic_file} already exists at {local_gz_path}')

        # Unzip the .gz file
        if not os.path.exists(local_fna_path):
            with gzip.open(local_gz_path, 'rb') as f_in:
                with open(local_fna_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            print(f'Unzipped: {local_gz_path} to {local_fna_path}')
        else:
            print(f'File {local_fna_path} already exists')

        # Concatenate sequences in the downloaded .fna file
        concatenated_fna_path = concatenate_fna_sequences(local_fna_path)

        # Secondary file download based on genome_id using ftplib
        ftp_host = 'ftp.bvbrc.org'
        ftp_path = f'/genomes/{genome_id}/{genome_id}.fna'
        secondary_local_path = os.path.join(local_dir, f'{genome_id}.fna')

        # Download the secondary file using ftplib
        if not os.path.exists(secondary_local_path):
            ftp = FTP(ftp_host)
            ftp.login()  # Anonymous login
            with open(secondary_local_path, 'wb') as f:
                ftp.retrbinary(f'RETR {ftp_path}', f.write)
            ftp.quit()
            print(f'Downloaded secondary file: {genome_id}.fna to {secondary_local_path}')
        else:
            print(f'Secondary file {genome_id}.fna already exists at {secondary_local_path}')

        # Concatenate sequences in the secondary .fna file
        concatenated_secondary_fna_path = concatenate_fna_sequences(secondary_local_path)

        # Run BLAST
        blast_output = os.path.join(local_dir, f'{genome_id}.blast')
        blast_command = [
            '/Users/niteshku/Downloads/blast/bin/blastn',
            '-query', concatenated_fna_path,
            '-subject', concatenated_secondary_fna_path,
            '-out', blast_output,
            '-outfmt', '6 qseqid sseqid pident length mismatch gaps qstart qend sstart send evalue bitscore qcovs qcovhsp qlen slen'
        ]

        subprocess.run(blast_command, check=True)
        print(f'BLAST output written to {blast_output}')
    else:
        print(f'No _genomic.fna.gz file found (excluding cds/rna) in {ftp_url}')

# Iterate over the DataFrame and download and process the files for each FTP path and genome_id
for _, row in df.iterrows():
    download_and_process_files(row['ftp_paths'], row['genome_id'])


/Users/niteshku/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Downloaded: GCF_949281945.1_PT33-3_genomic.fna.gz to /Users/niteshku/Notebooks/metagenomics/grant/GCF_949281945.1_PT33-3_genomic.fna.gz
Unzipped: /Users/niteshku/Notebooks/metagenomics/grant/GCF_949281945.1_PT33-3_genomic.fna.gz to /Users/niteshku/Notebooks/metagenomics/grant/GCF_949281945.1_PT33-3_genomic.fna
Concatenated sequence written to /Users/niteshku/Notebooks/metagenomics/grant/GCF_949281945.1_PT33-3_genomic_concatenated.fna
Downloaded secondary file: 24.8.fna to /Users/niteshku/Notebooks/metagenomics/grant/24.8.fna
Concatenated sequence written to /Users/niteshku/Notebooks/metagenomics/grant/24.8_concatenated.fna
BLAST output written to /Users/niteshku/Notebooks/metagenomics/grant/24.8.blast
No _genomic.fna.gz file found (excluding cds/rna) in https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/008/685/GCF_000008685.1_ASM868v1/
